# **Установка библиотек**

In [ ]:
pip install gensim pandas nltk pyldavis matplotlib

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
pip install pymorphy3 pymorphy3-dicts-ru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.7 MB/s eta 0:00:00


In [ ]:
!pip uninstall numpy pandas gensim scikit-learn -y
!pip install numpy==1.23.5 pandas gensim scikit-learn

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Using cached pandas-2.2.3-c

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from pymorphy3 import MorphAnalyzer
from gensim import corpora, models
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from string import punctuation

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


True

**Загрузка данных**

In [ ]:
# Загрузка данных из CSV файла
df = pd.read_csv('/content/lda.csv', sep=';')

df

,ОТВЕТЫ
0,"Первое - это информативность: понимание, что е..."
1,"Чего точно не хватает, так это совместного изу..."
2,Интерес родителей - о количестве свободных мес...
3,Для начала - определить родившегося ребёнка в ...
4,Родителей всегда интересует безопасность - нач...
5,Если родитель хочет отправить ребёнка в детски...
6,"Есть место, где семья живёт, и есть сады, кото..."
7,Сервис должен информировать о возрастных групп...
8,"Лучше, чтобы у родителей была возможность конт..."


# **Предобработка данных**

In [ ]:
one_string = ' '.join(df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1))

print(one_string)

Первое - это информативность: понимание, что есть рядом, что может помочь ребёнку. Информации много, но поиск сведений о графике, дисциплинах и т. д. нужно постоянно осуществлять самому и с нуля. Также внутри детских садов много своих курсов. Нужен сервис, который будет работать в формате одного окна и предоставлять информацию об учреждениях, находящихся в пешей доступности (500, 600, 700 метров). Некоторые родители готовы оплачивать коммерческие детские сады - о них информация так же должна быть. У каждой группы в детском саду есть самоорганизованный Telegram-канал. Был бы полезен сервис, который позволял бы осуществлять взаимодействие подобного характера - разделение по группам, возможность обратиться к заведующему. С одной стороны, этого все боятся - обращений может быть много. С другой стороны, заведующий может таким образом отследить «боли» родителей и систематизировать их. Чего точно не хватает, так это совместного изучения ребёнком и родителем какой-то дисциплины. Родителям пока

In [ ]:
#Токенизация текста
tokens = word_tokenize(one_string)
print(tokens)

['Первое', '-', 'это', 'информативность', ':', 'понимание', ',', 'что', 'есть', 'рядом', ',', 'что', 'может', 'помочь', 'ребёнку', '.', 'Информации', 'много', ',', 'но', 'поиск', 'сведений', 'о', 'графике', ',', 'дисциплинах', 'и', 'т.', 'д.', 'нужно', 'постоянно', 'осуществлять', 'самому', 'и', 'с', 'нуля', '.', 'Также', 'внутри', 'детских', 'садов', 'много', 'своих', 'курсов', '.', 'Нужен', 'сервис', ',', 'который', 'будет', 'работать', 'в', 'формате', 'одного', 'окна', 'и', 'предоставлять', 'информацию', 'об', 'учреждениях', ',', 'находящихся', 'в', 'пешей', 'доступности', '(', '500', ',', '600', ',', '700', 'метров', ')', '.', 'Некоторые', 'родители', 'готовы', 'оплачивать', 'коммерческие', 'детские', 'сады', '-', 'о', 'них', 'информация', 'так', 'же', 'должна', 'быть', '.', 'У', 'каждой', 'группы', 'в', 'детском', 'саду', 'есть', 'самоорганизованный', 'Telegram-канал', '.', 'Был', 'бы', 'полезен', 'сервис', ',', 'который', 'позволял', 'бы', 'осуществлять', 'взаимодействие', 'подоб

In [ ]:
punctuations = list(punctuation)
print(punctuations)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [ ]:
tokens_without_punct = [i for i in tokens if i not in punctuations]
print(tokens_without_punct)

['Первое', 'это', 'информативность', 'понимание', 'что', 'есть', 'рядом', 'что', 'может', 'помочь', 'ребёнку', 'Информации', 'много', 'но', 'поиск', 'сведений', 'о', 'графике', 'дисциплинах', 'и', 'т.', 'д.', 'нужно', 'постоянно', 'осуществлять', 'самому', 'и', 'с', 'нуля', 'Также', 'внутри', 'детских', 'садов', 'много', 'своих', 'курсов', 'Нужен', 'сервис', 'который', 'будет', 'работать', 'в', 'формате', 'одного', 'окна', 'и', 'предоставлять', 'информацию', 'об', 'учреждениях', 'находящихся', 'в', 'пешей', 'доступности', '500', '600', '700', 'метров', 'Некоторые', 'родители', 'готовы', 'оплачивать', 'коммерческие', 'детские', 'сады', 'о', 'них', 'информация', 'так', 'же', 'должна', 'быть', 'У', 'каждой', 'группы', 'в', 'детском', 'саду', 'есть', 'самоорганизованный', 'Telegram-канал', 'Был', 'бы', 'полезен', 'сервис', 'который', 'позволял', 'бы', 'осуществлять', 'взаимодействие', 'подобного', 'характера', 'разделение', 'по', 'группам', 'возможность', 'обратиться', 'к', 'заведующему', 

In [ ]:
low_tokens = [i.lower() for i in tokens_without_punct]
print(low_tokens)

['первое', 'это', 'информативность', 'понимание', 'что', 'есть', 'рядом', 'что', 'может', 'помочь', 'ребёнку', 'информации', 'много', 'но', 'поиск', 'сведений', 'о', 'графике', 'дисциплинах', 'и', 'т.', 'д.', 'нужно', 'постоянно', 'осуществлять', 'самому', 'и', 'с', 'нуля', 'также', 'внутри', 'детских', 'садов', 'много', 'своих', 'курсов', 'нужен', 'сервис', 'который', 'будет', 'работать', 'в', 'формате', 'одного', 'окна', 'и', 'предоставлять', 'информацию', 'об', 'учреждениях', 'находящихся', 'в', 'пешей', 'доступности', '500', '600', '700', 'метров', 'некоторые', 'родители', 'готовы', 'оплачивать', 'коммерческие', 'детские', 'сады', 'о', 'них', 'информация', 'так', 'же', 'должна', 'быть', 'у', 'каждой', 'группы', 'в', 'детском', 'саду', 'есть', 'самоорганизованный', 'telegram-канал', 'был', 'бы', 'полезен', 'сервис', 'который', 'позволял', 'бы', 'осуществлять', 'взаимодействие', 'подобного', 'характера', 'разделение', 'по', 'группам', 'возможность', 'обратиться', 'к', 'заведующему', 

In [ ]:
stemmer = SnowballStemmer('russian')
morph = MorphAnalyzer()

In [ ]:
stopwords = nltk.corpus.stopwords.words('russian')
print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [ ]:
lemms = [morph.parse(i)[0].normal_form for i in low_tokens]
print(lemms)

['первый', 'это', 'информативность', 'понимание', 'что', 'есть', 'рядом', 'что', 'мочь', 'помочь', 'ребёнок', 'информация', 'много', 'но', 'поиск', 'сведение', 'о', 'графика', 'дисциплина', 'и', 'т.', 'д.', 'нужно', 'постоянно', 'осуществлять', 'сам', 'и', 'с', 'нуль', 'также', 'внутри', 'детский', 'сад', 'много', 'свой', 'курс', 'нужный', 'сервис', 'который', 'быть', 'работать', 'в', 'формат', 'один', 'окно', 'и', 'предоставлять', 'информация', 'о', 'учреждение', 'находиться', 'в', 'пеший', 'доступность', '500', '600', '700', 'метр', 'некоторый', 'родитель', 'готовый', 'оплачивать', 'коммерческий', 'детский', 'сад', 'о', 'они', 'информация', 'так', 'же', 'должный', 'быть', 'у', 'каждый', 'группа', 'в', 'детский', 'сад', 'есть', 'самоорганизовать', 'telegram-канал', 'быть', 'бы', 'полезный', 'сервис', 'который', 'позволять', 'бы', 'осуществлять', 'взаимодействие', 'подобный', 'характер', 'разделение', 'по', 'группа', 'возможность', 'обратиться', 'к', 'заведовать', 'с', 'один', 'сторона

In [ ]:
words_without_stop = [i for i in lemms if i not in stopwords]
print(words_without_stop)

['первый', 'это', 'информативность', 'понимание', 'рядом', 'мочь', 'помочь', 'ребёнок', 'информация', 'поиск', 'сведение', 'графика', 'дисциплина', 'т.', 'д.', 'нужно', 'постоянно', 'осуществлять', 'нуль', 'также', 'внутри', 'детский', 'сад', 'свой', 'курс', 'нужный', 'сервис', 'который', 'работать', 'формат', 'окно', 'предоставлять', 'информация', 'учреждение', 'находиться', 'пеший', 'доступность', '500', '600', '700', 'метр', 'некоторый', 'родитель', 'готовый', 'оплачивать', 'коммерческий', 'детский', 'сад', 'информация', 'должный', 'каждый', 'группа', 'детский', 'сад', 'самоорганизовать', 'telegram-канал', 'полезный', 'сервис', 'который', 'позволять', 'осуществлять', 'взаимодействие', 'подобный', 'характер', 'разделение', 'группа', 'возможность', 'обратиться', 'заведовать', 'сторона', 'это', 'всё', 'бояться', 'обращение', 'мочь', 'сторона', 'заведовать', 'мочь', 'образ', 'отследить', '«', 'боль', '»', 'родитель', 'систематизировать', 'точно', 'хватать', 'это', 'совместный', 'изучени

In [ ]:
# Список токенов
tokens = words_without_stop

# Список стоп-слов
stop_words = ['городской', 'цифровой', 'сервис', 'должный', 'нужно', 'нужный', 'свой', 'видеть', 'считать', 'смотровой', 'это', 'площадка', 'удовлетворить', 'потребность', 'например', 'приложение', 'разработчик', 'несколько', 'который', 'некоторый', 'каждый', 'всё', 'быть', 'иметь', 'мочь','любой', 'предоставлять', 'предлагать', 'также', 'включая', 'любой', 'ребёнок', 'родитель', 'детский', 'сад', 'т.', 'д.', 'п.', 'подобный', 'точно', '«', '»', 'что-то', 'ещё', 'особенно', 'скорее', 'поэтому', 'такой-то', 'весь', 'хотеть', 'помимо', 'возможно', 'какой-то', 'почему', 'либо', 'среди', 'конкретный', 'некий', 'вообще', 'разный', 'сколько', 'прямо', 'сразу', 'всё-таки', 'немножко', 'обязательно', 'обязательный', 'второй', 'дошкольник', 'первый', 'осуществлять', 'нуль', 'внутри', 'постоянно', '500', '600', '700', 'готовый', 'позволять', 'хватать', 'показывать', 'определить', 'говорить', 'происходить', 'большинство', 'далее', 'предположим', 'эгс', 'давать', 'просто', 'оказаться', 'конкретно', 'учреждение', 'понимать', 'садик']

# Удаление стоп-слов из списка токенов
filtered_tokens = [token for token in tokens if token not in stop_words]

print(filtered_tokens)

['информативность', 'понимание', 'рядом', 'помочь', 'информация', 'поиск', 'сведение', 'графика', 'дисциплина', 'курс', 'работать', 'формат', 'окно', 'информация', 'находиться', 'пеший', 'доступность', 'метр', 'оплачивать', 'коммерческий', 'информация', 'группа', 'самоорганизовать', 'telegram-канал', 'полезный', 'взаимодействие', 'характер', 'разделение', 'группа', 'возможность', 'обратиться', 'заведовать', 'сторона', 'бояться', 'обращение', 'сторона', 'заведовать', 'образ', 'отследить', 'боль', 'систематизировать', 'совместный', 'изучение', 'дисциплина', 'результат', 'обучение', 'поделка', 'рисунок', 'страна', 'отсутствовать', 'практика', 'вовлечение', 'процесс', 'дошкольный', 'образование', 'интерес', 'количество', 'свободный', 'место', 'близкий', 'дом', 'район', 'актуально', 'семья', 'ходить', 'место', 'жительство', 'хватить', 'место', 'интересовать', 'отзыв', 'персонал', 'развивать', 'программа', 'начало', 'родиться', 'информировать', 'процесс', 'зачисление', 'комплектование', 'поп

In [ ]:
doc_size = 8
docs = [filtered_tokens[i:i + doc_size] for i in range(0, len(filtered_tokens), doc_size)]
docs

[['информативность',
  'понимание',
  'рядом',
  'помочь',
  'информация',
  'поиск',
  'сведение',
  'графика'],
 ['дисциплина',
  'курс',
  'работать',
  'формат',
  'окно',
  'информация',
  'находиться',
  'пеший'],
 ['доступность',
  'метр',
  'оплачивать',
  'коммерческий',
  'информация',
  'группа',
  'самоорганизовать',
  'telegram-канал'],
 ['полезный',
  'взаимодействие',
  'характер',
  'разделение',
  'группа',
  'возможность',
  'обратиться',
  'заведовать'],
 ['сторона',
  'бояться',
  'обращение',
  'сторона',
  'заведовать',
  'образ',
  'отследить',
  'боль'],
 ['систематизировать',
  'совместный',
  'изучение',
  'дисциплина',
  'результат',
  'обучение',
  'поделка',
  'рисунок'],
 ['страна',
  'отсутствовать',
  'практика',
  'вовлечение',
  'процесс',
  'дошкольный',
  'образование',
  'интерес'],
 ['количество',
  'свободный',
  'место',
  'близкий',
  'дом',
  'район',
  'актуально',
  'семья'],
 ['ходить',
  'место',
  'жительство',
  'хватить',
  'место',
  'и

In [ ]:
from gensim.models import Phrases

In [ ]:
stemmer = SnowballStemmer("russian")
morph = MorphAnalyzer()

In [ ]:
stemmed_docs = []
stem_to_lemma = defaultdict(set)

for doc in docs:
    stemmed_doc = []
    for token in doc:
        stem = stemmer.stem(token)
        lemma = morph.parse(token)[0].normal_form
        stemmed_doc.append(stem)
        stem_to_lemma[stem].add(lemma)
    stemmed_docs.append(stemmed_doc)

In [ ]:
dictionary = Dictionary(docs)
dictionary.filter_extremes(
    no_below=4,
    no_above=0.3,
    keep_n=1000
)

# **Тематическое моделирование**

In [ ]:
model = LdaModel(
    corpus=[dictionary.doc2bow(doc) for doc in docs],
    id2word=dictionary,
    num_topics=7,
    passes=20,
    alpha=0.1,
    eta=0.01,
    random_state=42
)

In [ ]:
for topic_id in range(model.num_topics):
    words = [word for word, _ in model.show_topic(topic_id, 8)]
    print(f"Тема {topic_id+1}: {', '.join(words)}")

Тема 1: место, заявление, группа, образовательный, информация, возможность, информировать, портал
Тема 2: портал, процесс, заявление, возможность, место, информация, образовательный, информировать
Тема 3: возможность, образовательный, информация, портал, заявление, место, информировать, процесс
Тема 4: информация, место, возможность, заявление, образовательный, портал, процесс, группа
Тема 5: информировать, процесс, возможность, образовательный, заявление, место, группа, информация
Тема 6: информация, заявление, образовательный, возможность, место, портал, процесс, группа
Тема 7: группа, информация, место, возможность, образовательный, заявление, процесс, информировать
